# 0. Libs

In [2]:
from spacy import load # lib para carregar o modelo em português
from spacy.matcher import Matcher # lib para criar regras para detectar palavras positivas e negativas
from spacy.tokens import Doc # lib para armazenar o sentimento em doc 

# 1. Carregar o modelo para o idioma português (pt_core_news_sm)

In [3]:
# !pip install spacy

In [4]:
# !python -m spacy download pt_core_news_sm

In [5]:
nlp = load("pt_core_news_sm") # Modelo "pequeno" de notícias que será usado para análise de sentimentos

# 2. Criar regras com Matcher para detectar palavras positivas e negativas

In [6]:
# Detecção de sentimentos
matcher = Matcher(nlp.vocab)

# Palavras positivas
positive_patterns = [
    [{"LOWER": "bom"}],
    [{"LOWER": "ótimo"}],
    [{"LOWER": "excelente"}],
    [{"LOWER": "adorei"}],
    [{"LOWER": "incrível"}],
    [{"LOWER": "amei"}],
    [{"LOWER": "feliz"}],
    [{"LOWER": "alegre"}],
    [{"LOWER": "maravilhoso"}],
    [{"LOWER": "superou"}, {"LOWER": "expectativas"}],
]
matcher.add("POSITIVE", positive_patterns)

# Palavras negativas
negative_patterns = [
    [{"LOWER": "ruim"}],
    [{"LOWER": "péssimo"}],
    [{"LOWER": "detestei"}],
    [{"LOWER": "odeio"}],
    [{"LOWER": "triste"}],
    [{"LOWER": "decepcionante"}],
    [{"LOWER": "horrível"}],
    [{"LOWER": "lento"}],
    [{"LOWER": "bug"}],
    [{"LOWER": "arrependi"}],
]
matcher.add("NEGATIVE", negative_patterns)

# 3. Adicionar uma extensão personalizada ao objeto Doc para armazenar o sentimento detectado

In [7]:
# Criação de doc para armazenar os sentimentos
Doc.set_extension("sentiment", default=None, force=True)

# Função para analisar o sentimento de um Doc
def analyze_sentiment(doc):
    positive_matches = 0
    negative_matches = 0
    matches = matcher(doc)
    for match_id, start, end in matches:
        if nlp.vocab.strings[match_id] == "POSITIVE":
            positive_matches += 1
        elif nlp.vocab.strings[match_id] == "NEGATIVE":
            negative_matches += 1

    if positive_matches > negative_matches:
        doc._.sentiment = "Positivo"
    elif negative_matches > positive_matches:
        doc._.sentiment = "Negativo"
    elif positive_matches > 0 or negative_matches > 0:
        doc._.sentiment = "Neutro/Misto"
    else:
        doc._.sentiment = "Não detectado"
    return doc._.sentiment

# 4. Analisar uma lista com pelo menos 5 frases e imprimir o sentimento de cada uma

In [8]:
frases = [
    "A comida estava muito boa e o atendimento foi excelente.",
    "Não gostei nada do filme, achei péssimo e lento.",
    "O produto chegou rápido, mas a embalagem estava um pouco amassada.",
    "Adorei a nova atualização do sistema, tudo ficou mais fluido.",
    "Me arrependi da compra, o aparelho apresentou um bug constante.",
    "O dia hoje está maravilhoso e ensolarado.",
    "Achei o preço um pouco alto, mas o produto cumpre a função regular.",
]

print("Análise de Sentimento das Frases:")
for frase in frases:
    doc = nlp(frase)
    sentimento = analyze_sentiment(doc)
    print(f"Frase: '{frase}' - Sentimento: {sentimento}")

Análise de Sentimento das Frases:
Frase: 'A comida estava muito boa e o atendimento foi excelente.' - Sentimento: Positivo
Frase: 'Não gostei nada do filme, achei péssimo e lento.' - Sentimento: Negativo
Frase: 'O produto chegou rápido, mas a embalagem estava um pouco amassada.' - Sentimento: Não detectado
Frase: 'Adorei a nova atualização do sistema, tudo ficou mais fluido.' - Sentimento: Positivo
Frase: 'Me arrependi da compra, o aparelho apresentou um bug constante.' - Sentimento: Negativo
Frase: 'O dia hoje está maravilhoso e ensolarado.' - Sentimento: Positivo
Frase: 'Achei o preço um pouco alto, mas o produto cumpre a função regular.' - Sentimento: Não detectado
